# Load Data

In [1]:
import json
import torch
import random
import os
import pickle
import collections
from sklearn.preprocessing import LabelEncoder
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from torch import nn
import numpy as np
from scipy.linalg import inv
from torch.optim import Adam
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import re
from collections import Counter

In [2]:
seed_value=883070
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

# MTOP

In [3]:
# # Function to read and parse the MTOP file
# def read_mtop_file(file_path):
#     data = []
#     with open(file_path, 'r', encoding='utf-8') as file:
#         for line in file:
#             fields = line.strip().split('\t')
#             if len(fields) < 8:
#                 continue  # Skip any malformed lines
#             record = {
#                 'ID': fields[0],
#                 'Intent': fields[1],
#                 'Utterance': fields[3],
#                 'Domain': fields[4]
#             }
#             data.append(record)
#     return data
    
# # Function to randomly select specified number of OOD domains
# def select_ood_domains(domains, num_ood):
#     return random.sample(sorted(list(domains)), num_ood)

# # Configuration parameters
# english_dir = 'mtop/en'
# num_ood_domains = 2  # Number of domains to treat as OOD

# # Read dataset
# test_data = read_mtop_file(f'{english_dir}/test.txt')
# train_data = read_mtop_file(f'{english_dir}/train.txt')
# eval_data = read_mtop_file(f'{english_dir}/eval.txt')

# # Gather all domains and select OOD domains
# domains = set([record['Domain'] for record in train_data + test_data + eval_data])
# ood_domains = select_ood_domains(domains, num_ood_domains)

# # Initialize data structures for sentences and labels
# train_sentences, train_labels = [], []
# val_sentences, val_labels = [], []
# test_sentences, test_labels = [], []
# oos_val_sentences, oos_test_sentences = [], []

# # Distribute data based on OOD selection
# for dataset in [(train_data, train_sentences, train_labels), (eval_data, val_sentences, val_labels), (test_data, test_sentences, test_labels)]:
#     for record in dataset[0]:
#         if record['Domain'] in ood_domains:
#             if dataset is eval_data:
#                 oos_val_sentences.append(record['Utterance'])
#             else:
#                 oos_test_sentences.append(record['Utterance'])
#         else:
#             dataset[1].append(record['Utterance'])
#             dataset[2].append(record['Domain'])

# # Rebalance OOS data between validation and test
# combined_oos = oos_val_sentences + oos_test_sentences
# random.shuffle(combined_oos)
# half_index = len(combined_oos) // 2
# oos_val_sentences = combined_oos[:half_index]
# oos_test_sentences = combined_oos[half_index:]
# model_name = "ce_model_bert_mtop.pth"
# # Check the distribution and the selected OOD domains
# (ood_domains, len(train_sentences), len(val_sentences), len(test_sentences), len(oos_val_sentences), len(oos_test_sentences))


In [4]:
def read_mtop_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            fields = line.strip().split('\t')
            if len(fields) < 8:
                continue  # Skip any malformed lines
            record = {
                'ID': fields[0],
                'Intent': fields[1],
                'Utterance': fields[3],
                'Domain': fields[4]
            }
            data.append(record)
    return data

def select_ood_domains(domains, num_ood):
    return ["timer"]  # Hardcoded OOD domains

# Configuration parameters
english_dir = 'mtop/en'

# Read dataset
all_data = read_mtop_file(f'{english_dir}/test.txt') + \
           read_mtop_file(f'{english_dir}/train.txt') + \
           read_mtop_file(f'{english_dir}/eval.txt')

domains = set(record['Domain'] for record in all_data)
ood_domains = select_ood_domains(domains, 1)  # Using 1 as hardcoded number of OOD domains

# Separate OOD data based on domain
in_domain_data = [record for record in all_data if record['Domain'] not in ood_domains]
ood_data = [record for record in all_data if record['Domain'] in ood_domains]

intent_counts = Counter(record['Intent'] for record in in_domain_data)

# Filter intents with at least 10 instances
sufficient_data = [record for record in in_domain_data if intent_counts[record['Intent']] > 10]

train_val_data, test_data = train_test_split(
    sufficient_data, test_size=0.2, random_state=seed_value, stratify=[record['Intent'] for record in sufficient_data]
)
train_data, val_data = train_test_split(
    train_val_data, test_size=0.125, random_state=seed_value, stratify=[record['Intent'] for record in train_val_data]
)

# Split OOD data between validation and test
oos_val_data, oos_test_data = train_test_split(ood_data, test_size=0.67, random_state=seed_value)  # No stratification here since it's all OOD

# Extract sentences and labels
train_sentences = [record['Utterance'] for record in train_data]
train_labels = [record['Intent'] for record in train_data]

val_sentences = [record['Utterance'] for record in val_data]
val_labels = [record['Intent'] for record in val_data]

test_sentences = [record['Utterance'] for record in test_data]
test_labels = [record['Intent'] for record in test_data]

oos_val_sentences = [record['Utterance'] for record in oos_val_data]
oos_test_sentences = [record['Utterance'] for record in oos_test_data]

# Summary of splits
summary = {
    "OOD Domains": ood_domains,
    "Train Set Size": len(train_sentences),
    "Validation Set Size": len(val_sentences),
    "Test Set Size": len(test_sentences),
    "OOS Validation Set Size": len(oos_val_sentences),
    "OOS Test Set Size": len(oos_test_sentences)
}




model_name = "ce_model_bert_mtop.pth"
summary

{'OOD Domains': ['timer'],
 'Train Set Size': 14465,
 'Validation Set Size': 2067,
 'Test Set Size': 4134,
 'OOS Validation Set Size': 491,
 'OOS Test Set Size': 997}

# 20NewsGroup

In [5]:
# from sklearn.datasets import fetch_20newsgroups

# # Load the dataset
# newsgroups = fetch_20newsgroups(subset='all', shuffle=True, random_state=seed_value)
# def extract_and_clean_subject(text):
#     # Search for the subject line
#     match = re.search(r'Subject: (.+)', text)
#     if not match:
#         return "no subject found"
    
#     # Extract the subject and remove leading "Re: " if present
#     subject = match.group(1)
#     cleaned_subject = re.sub(r'^\s*Re:\s*', '', subject, flags=re.IGNORECASE)
    
#     # Return the cleaned subject in lowercase
#     return cleaned_subject.lower()
# # Data and labels
# data = newsgroups.data
# data = [extract_and_clean_subject(doc) for doc in data]
# labels = newsgroups.target
# target_names = newsgroups.target_names

# # Define OOD categories (any category that includes 'misc')
# ood_categories = [name for name in target_names if 'misc.forsale' in name]
# ood_indices = [target_names.index(cat) for cat in ood_categories]
# id_indices = [i for i in range(len(target_names)) if i not in ood_indices]

# # Filter ID and OOD data
# id_data = [data[i] for i in range(len(data)) if labels[i] in id_indices]
# id_labels = [labels[i] for i in range(len(data)) if labels[i] in id_indices]
# ood_data = [data[i] for i in range(len(data)) if labels[i] in ood_indices]

# # Split ID data into train, validation, and test
# train_sentences, temp_data, train_labels, temp_labels = train_test_split(
#     id_data, id_labels, test_size=0.3, random_state=seed_value)
# val_sentences, test_sentences, val_labels, test_labels = train_test_split(
#     temp_data, temp_labels, test_size=0.5, random_state=seed_value)

# # Assume we use the OOD data only for validation and testing
# oos_val_sentences, oos_test_sentences = train_test_split(ood_data, test_size=0.5, random_state=42)
# model_name = "ce_model_bert_20newsgroup.pth"

# SearchSnippets

In [6]:
# # Path to the files
# text_path = 'search_snippets/SearchSnippets.txt'
# label_path = 'search_snippets/SearchSnippets_label.txt'

# # Load sentences and labels
# with open(text_path, 'r', encoding='utf-8') as file:
#     sentences = file.readlines()
# with open(label_path, 'r', encoding='utf-8') as file:
#     labels = [int(line.strip()) for line in file.readlines()]

# # Split data into training and temporary (val + test)
# train_sentences, temp_sentences, train_labels, temp_labels = train_test_split(
#     sentences, labels, test_size=0.35, random_state=seed_value)

# # Split temporary data into validation and test
# val_sentences, test_sentences, val_labels, test_labels = train_test_split(
#     temp_sentences, temp_labels, test_size=0.5, random_state=seed_value)

# # Randomly select 2 classes as OOD classes, rest as ID classes
# all_classes = list(range(1, 9))  # Assuming classes are labeled from 1 to 8
# ood_classes = set(random.sample(all_classes, 2))
# id_classes = set(all_classes) - ood_classes

# # Function to filter sentences by class
# def filter_by_class(sentences, labels, classes):
#     return [s for s, l in zip(sentences, labels) if l in classes]

# # Populate ID and OOD datasets
# id_train_sentences = filter_by_class(train_sentences, train_labels, id_classes)
# id_train_labels = [l for l in train_labels if l in id_classes]
# id_val_sentences = filter_by_class(val_sentences, val_labels, id_classes)
# id_val_labels = [l for l in val_labels if l in id_classes]
# id_test_sentences = filter_by_class(test_sentences, test_labels, id_classes)
# id_test_labels = [l for l in test_labels if l in id_classes]

# oos_val_sentences = filter_by_class(val_sentences, val_labels, ood_classes)
# oos_test_sentences = filter_by_class(test_sentences, test_labels, ood_classes)

# # Assign the filtered lists back to the main variables
# train_sentences, train_labels = id_train_sentences, id_train_labels
# val_sentences, val_labels = id_val_sentences, id_val_labels
# test_sentences, test_labels = id_test_sentences, id_test_labels
# model_name = "ce_model_bert_search_snippets.pth"

# Cerence

In [7]:
# ood_suffix = "just_talk_embedded_reject_twitter_blind-testonly.en-US.NCS5.1.final_1.input"
# def load_sentences_and_labels(data_dir, subset, in_system=True, suffix=".input"):
#     """
#     Load sentences and their labels from a given directory.
    
#     Parameters:
#     - data_dir: Base directory containing the data subsets ('dev', 'test', 'train').
#     - subset: Subset of data to load ('dev', 'train', 'test').
#     - in_system: Boolean indicating whether to load InSys (True) or OOS (False) data.
#     - suffix: File suffix to look for.
    
#     Returns:
#     - sentences: List of sentences loaded from the files.
#     - labels: List of corresponding labels derived from file names.
#     """
#     sentences = []
#     labels = []
#     subset_dir = "InSys" if in_system else "OOS"
#     full_path = os.path.join(data_dir, subset, subset_dir)
    
#     for filename in os.listdir(full_path):
#         if filename.endswith(suffix):
#             label = filename.replace(suffix, '')
#             with open(os.path.join(full_path, filename), 'r', encoding='utf-8') as file:
#                 file_sentences = file.readlines()
#                 sentences.extend([sentence.strip() for sentence in file_sentences])
#                 labels.extend([label] * len(file_sentences))
    
#     return sentences, labels

# # Specify the base directory where your 'dev', 'test', 'train' directories are located
# base_dir = 'cerence_data'

# # Load the datasets
# train_sentences, train_labels = load_sentences_and_labels(base_dir, 'train', suffix=".train.input")
# val_sentences, val_labels = load_sentences_and_labels(base_dir, 'dev', suffix=".dev.input")
# test_sentences, test_labels = load_sentences_and_labels(base_dir, 'test', in_system=True, suffix=".test.input")
# oos_test_sentences, _ = load_sentences_and_labels(base_dir, 'test', in_system=False, suffix=ood_suffix)

# # Split half as val
# random.shuffle(oos_test_sentences)
# split_index = len(oos_test_sentences) // 2

# # Split the sentences into two halves
# oos_val_sentences = oos_test_sentences[:split_index]
# oos_test_sentences = oos_test_sentences[split_index:]

# train_sentences, _, train_labels, _ = train_test_split(
#     train_sentences, train_labels, train_size=1/25, stratify=train_labels, random_state=seed_value)
# test_sentences, _, test_labels, _ = train_test_split(
#     test_sentences, test_labels, train_size=1/20, stratify=test_labels, random_state=seed_value)

# model_name = "ce_model_bert_cerence.pth"

# Load CLINC150

In [8]:
# # Load the dataset
# with open("clinc150_uci/data_full.json", "r") as file:
#     data = json.load(file)
# # Extracting data
# train_data = data['train']
# val_data = data['val']
# test_data = data['test']

# oos_train_data = data['oos_train']
# oos_val_data = data['oos_val']
# oos_test_data = data['oos_test']

# # Get sentences and labels
# train_sentences = [item[0] for item in train_data]
# train_labels = [item[1] for item in train_data]

# val_sentences = [item[0] for item in val_data]
# val_labels = [item[1] for item in val_data]

# test_sentences = [item[0] for item in test_data]
# test_labels = [item[1] for item in test_data]

# oos_train_sentences = [item[0] for item in oos_train_data]
# oos_val_sentences = [item[0] for item in oos_val_data]
# oos_test_sentences = [item[0] for item in oos_test_data]
# model_name = "ce_model_bert_CLINC150.pth"

# BANKING77-OOS

In [9]:
# # Define the IntentExample class and load_intent_examples function as provided
# class IntentExample:
#     def __init__(self, text, label, do_lower_case):
#         self.original_text = text
#         self.text = text
#         self.label = label
#         if do_lower_case:
#             self.text = self.text.lower()

# def load_intent_examples(file_path, do_lower_case=True):
#     examples = []
#     with open(f'{file_path}/seq.in', 'r', encoding="utf-8") as f_text, open(f'{file_path}/label', 'r', encoding="utf-8") as f_label:
#         for text, label in zip(f_text, f_label):
#             e = IntentExample(text.strip(), label.strip(), do_lower_case)
#             examples.append(e)
#     return examples

# # Define paths to the dataset directories
# base_dir = 'Few-Shot-Intent-Detection/Datasets/BANKING77-OOS'
# paths = {
#     'train': f'{base_dir}/train',
#     'valid': f'{base_dir}/valid',
#     'test': f'{base_dir}/test',
#     'oos_val': f'{base_dir}/ood-oos/valid',
#     'oos_test': f'{base_dir}/ood-oos/test'
# }
# datasets = {key: load_intent_examples(path) for key, path in paths.items()}

# # Extract sentences and labels from the loaded datasets
# train_sentences = [e.text for e in datasets['train']]
# train_labels = [e.label for e in datasets['train']]

# val_sentences = [e.text for e in datasets['valid']]
# val_labels = [e.label for e in datasets['valid']]

# test_sentences = [e.text for e in datasets['test']]
# test_labels = [e.label for e in datasets['test']]

# oos_val_sentences = [e.text for e in datasets['oos_val']]
# oos_test_sentences = [e.text for e in datasets['oos_test']]
# model_name = "ce_model_bert_BANKING77.pth"


# SNIPS

In [10]:
# # Define the IntentExample class and load_intent_examples function as provided
# class IntentExample:
#     def __init__(self, text, label, do_lower_case):
#         self.original_text = text
#         self.text = text
#         self.label = label
#         if do_lower_case:
#             self.text = self.text.lower()

# def load_intent_examples(file_path, do_lower_case=True):
#     examples = []
#     with open(f'{file_path}/seq.in', 'r', encoding="utf-8") as f_text, open(f'{file_path}/label', 'r', encoding="utf-8") as f_label:
#         for text, label in zip(f_text, f_label):
#             e = IntentExample(text.strip(), label.strip(), do_lower_case)
#             examples.append(e)
#     return examples
# # Define paths to the dataset directories
# base_dir = 'Few-Shot-Intent-Detection/Datasets/SNIPS'
# paths = {
#     'train': f'{base_dir}/train',
#     'valid': f'{base_dir}/valid',
#     'test': f'{base_dir}/test'
# }
# # Define paths to the dataset directories
# datasets = {key: load_intent_examples(path) for key, path in paths.items()}

# # Extract sentences and labels from the loaded datasets
# def extract_sentences_labels(dataset):
#     sentences = [e.text for e in dataset]
#     labels = [e.label for e in dataset]
#     return sentences, labels

# all_train_sentences, all_train_labels = extract_sentences_labels(datasets['train'])
# all_val_sentences, all_val_labels = extract_sentences_labels(datasets['valid'])
# all_test_sentences, all_test_labels = extract_sentences_labels(datasets['test'])

# # Count the frequency of each label in the training data
# label_freq = collections.Counter(all_train_labels)

# # Calculate the threshold for selecting 75% of the data as in-domain
# threshold = len(all_train_labels) * 0.75
# cumulative_count = 0
# selected_labels = []

# # Randomize the label order for fairness
# all_labels = list(label_freq.keys())
# random.shuffle(all_labels)

# for label in all_labels:
#     count = label_freq[label]
#     cumulative_count += count
#     selected_labels.append(label)
#     if cumulative_count >= threshold:
#         break

# # Function to filter data by selected labels
# def filter_data(sentences, labels, selected_labels):
#     filtered_data = [(s, l) for s, l in zip(sentences, labels) if l in selected_labels]
#     return [item[0] for item in filtered_data], [item[1] for item in filtered_data]

# # Filter the datasets
# train_sentences, train_labels = filter_data(all_train_sentences, all_train_labels, selected_labels)
# val_sentences, val_labels = filter_data(all_val_sentences, all_val_labels, selected_labels)
# test_sentences, test_labels = filter_data(all_test_sentences, all_test_labels, selected_labels)

# # Identify and separate the out-of-domain data
# oos_labels = set(all_labels) - set(selected_labels)
# oos_train_sentences, oos_train_labels = filter_data(all_train_sentences, all_train_labels, oos_labels)
# oos_val_sentences, oos_val_labels = filter_data(all_val_sentences, all_val_labels, oos_labels)
# oos_test_sentences, oos_test_labels = filter_data(all_test_sentences, all_test_labels, oos_labels)

# # Evenly split oos_train data between validation and test
# half_index = len(oos_train_sentences) // 2
# oos_val_sentences.extend(oos_train_sentences[:half_index])
# oos_val_labels.extend(oos_train_labels[:half_index])
# oos_test_sentences.extend(oos_train_sentences[half_index:])
# oos_test_labels.extend(oos_train_labels[half_index:])
# model_name = "ce_model_bert_SNIP.pth"
# # oos_scenario is no longer a single scenario but a list of OOD labels
# oos_scenarios = list(oos_labels)

# ROSTD

In [11]:
# from datasets import load_dataset

# dataset = load_dataset("cmaldona/Generalization-MultiClass-CLINC150-ROSTD", "rostd+")

# train_sentences = []
# train_labels = []
# val_sentences = []
# val_labels = []
# test_sentences = []
# test_labels = []
# oos_test_sentences = []

# # Extract training data
# for example in dataset['train']:
#     train_sentences.append(example['data'].lower())
#     train_labels.append(example['labels'])

# # Extract validation data
# for example in dataset['validation']:
#     val_sentences.append(example['data'].lower())
#     val_labels.append(example['labels'])

# # Extract test data and separate ID from OOS
# for example in dataset['test']:
#     if example['generalisation'] == 'ID':
#         test_sentences.append(example['data'].lower())
#         test_labels.append(example['labels'])
#     elif example['generalisation'] == 'near-OOD' or example['generalisation'] == 'far-OOD':# OOS
#         try:
#             oos_test_sentences.append(example['data'].lower())
#         except:
#             continue
            

# model_name = "ce_model_bert_ROSTD.pth"

# StackOverflow

In [12]:
# split_number = 1

# base_dir = 'stackoverflow_data'  

# # Construct the path to the specific split
# split_dir = os.path.join(base_dir, f'split{split_number}')

# # Function to load data from a .pkl file
# def load_data_from_split(split_dir, file_name):
#     with open(os.path.join(split_dir, file_name), 'rb') as file:
#         return pickle.load(file)

# # Load the datasets
# train_sentences = load_data_from_split(split_dir, 'train_sentences.pkl')
# train_labels = load_data_from_split(split_dir, 'train_labels.pkl')
# val_sentences = load_data_from_split(split_dir, 'val_sentences.pkl')
# val_labels = load_data_from_split(split_dir, 'val_labels.pkl')
# test_sentences = load_data_from_split(split_dir, 'test_sentences.pkl')
# test_labels = load_data_from_split(split_dir, 'test_labels.pkl')
# oos_val_sentences = load_data_from_split(split_dir, 'oos_val_sentences.pkl')
# oos_test_sentences = load_data_from_split(split_dir, 'oos_test_sentences.pkl')
# model_name = "ce_model_bert_Stack.pth"

In [13]:
model_name = f"{seed_value}_{model_name}"

# Encode Labels

In [14]:
label_encoder = LabelEncoder()
# Fit the label encoder and transform labels to integers
encoded_train_labels = label_encoder.fit_transform(train_labels)
encoded_val_labels = label_encoder.fit_transform(val_labels)
encoded_test_labels = label_encoder.fit_transform(test_labels)

# Tokenize our sentences and create Dataloaders

In [15]:
pretrained_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
transformer_model = AutoModel.from_pretrained(pretrained_model_name)
tokenized_lengths = [len(tokenizer.encode(sentence, add_special_tokens=True)) for sentence in train_sentences]
max_length = max(tokenized_lengths)
print(f"Max length for tokenizer: {max_length}")

class TextDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(sentences, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 2. Create the dataset
train_dataset = TextDataset(train_sentences, encoded_train_labels, tokenizer, max_length)
val_dataset = TextDataset(val_sentences, encoded_val_labels, tokenizer, max_length)
test_dataset = TextDataset(test_sentences, encoded_test_labels, tokenizer, max_length)

Max length for tokenizer: 41


# Define functions to encode our sentences

In [16]:
transformer_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer_model = transformer_model.to(device)
def encode_sentences(model, sentences, tokenizer=tokenizer, batch_size=256):
    model = model.to(device)
    sentence_embeddings = []

    # Process sentences in batches
    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i+batch_size]
        encoded_input = tokenizer(batch_sentences, return_tensors='pt', padding=True, truncation=True, max_length=512)
        
        # Move the batch to the same device as the model
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        
        with torch.no_grad():
            model_output = model(**encoded_input)

        pooled_output = model_output.last_hidden_state.mean(dim=1)
        sentence_embeddings.append(pooled_output)

    # Concatenate all batched embeddings and move to CPU in one go
    sentence_embeddings_np = torch.cat(sentence_embeddings, dim=0).cpu().numpy()
    
    return sentence_embeddings_np

# Define our model

In [17]:
transformer_model = AutoModel.from_pretrained(pretrained_model_name)
import torch.nn as nn

class TextClassifier(nn.Module):
    def __init__(self, transformer_model, num_labels):
        super(TextClassifier, self).__init__()
        self.transformer = transformer_model
        self.classifier = nn.Linear(self.transformer.config.hidden_size, num_labels)
    
    def forward(self, input_ids, attention_mask):
        # Get the output from the transformer model
        transformer_output = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        # Pool the outputs into a single sentence vector
        # You can use `transformer_output.last_hidden_state.mean(dim=1)` for mean pooling
        # or `transformer_output.last_hidden_state.max(dim=1).values` for max pooling
        sentence_embedding = transformer_output.last_hidden_state.max(dim=1).values
        # Forward pass through the classifier layer
        return self.classifier(sentence_embedding)


# Initiallize everything else we needed

In [18]:
unique_intents = list(set(train_labels)) 
transformer_model = AutoModel.from_pretrained(pretrained_model_name)
transformer_model.to(device)
model = TextClassifier(transformer_model, len(unique_intents))
model.to(device)
optimizer = Adam(model.parameters(), lr=2.25E-05)
training_losses = []
validation_losses = []
batch_size= 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
loss_function = nn.CrossEntropyLoss()

In [19]:
num_epochs = 10

# Training Loop!!!

In [ ]:
if not os.path.exists(model_name):
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        # Training Phase
        model.train()  # Set the model to training mode
        total_train_loss = 0
        for batch in train_dataloader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            optimizer.zero_grad()  # Zero the gradients
            predictions = model(input_ids, attention_mask)  # Forward pass
            loss = loss_function(predictions, labels) 
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
    
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_dataloader)
        training_losses.append(avg_train_loss) 
    
        # Validation Phase
        model.eval()  # Set the model to evaluation mode
        total_val_loss = 0
        with torch.no_grad():  # Disable gradient calculations
            for batch in val_dataloader:
                input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
                predictions = model(input_ids, attention_mask) 
                loss = loss_function(predictions, labels)  # Compute loss
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_dataloader)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            # Save the model
            torch.save(model, model_name)
            print(f"Epoch {epoch+1}/{num_epochs}: Lower validation loss found. Model saved.")
        validation_losses.append(avg_val_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.5e}, Validation Loss: {avg_val_loss:.5e}")
else:
        print("training skipped")

Epoch 1/10: Lower validation loss found. Model saved.
Epoch 1/10, Training Loss: 1.86166e+00, Validation Loss: 7.22584e-01
Epoch 2/10: Lower validation loss found. Model saved.
Epoch 2/10, Training Loss: 5.47760e-01, Validation Loss: 3.59231e-01
Epoch 3/10: Lower validation loss found. Model saved.
Epoch 3/10, Training Loss: 2.94018e-01, Validation Loss: 2.28093e-01
Epoch 4/10: Lower validation loss found. Model saved.
Epoch 4/10, Training Loss: 1.82920e-01, Validation Loss: 1.74819e-01
Epoch 5/10: Lower validation loss found. Model saved.
Epoch 5/10, Training Loss: 1.15361e-01, Validation Loss: 1.54268e-01
Epoch 6/10: Lower validation loss found. Model saved.
Epoch 6/10, Training Loss: 8.03761e-02, Validation Loss: 1.34471e-01


# Calculate means and covariance matrix

In [ ]:
fine_model = torch.load(model_name)
fine_model.eval()  # Put the model in evaluation mode
fine_model = fine_model.to(device)

In [ ]:
# get trasformer part of the model
fine_model = fine_model.transformer

In [ ]:
train_embeddings = encode_sentences(fine_model, train_sentences)
val_embeddings = encode_sentences(fine_model, val_sentences)
test_embeddings = encode_sentences(fine_model, test_sentences)
oos_val_embeddings = encode_sentences(fine_model, oos_val_sentences)
oos_test_embeddings = encode_sentences(fine_model, oos_test_sentences)

In [ ]:
intent_means = {}
for encoded_label in np.unique(encoded_train_labels):
    # Find indices where the encoded label matches
    indices = np.where(encoded_train_labels == encoded_label)[0]
    
    # Calculate the mean embedding for the current intent
    intent_embeddings = train_embeddings[indices]
    intent_mean = np.mean(intent_embeddings, axis=0)
    
    # Use the encoded label as the dictionary key
    intent_means[encoded_label] = intent_mean

In [ ]:
covariance = np.cov(train_embeddings, rowvar=False)
cov_inverse = inv(covariance)

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
from scipy.spatial import distance
from sklearn.metrics import average_precision_score

In [ ]:
def min_mahalanobis_for_sample(sample, intent_means, cov_inverse):
    distances = [distance.mahalanobis(sample, mean, cov_inverse) for mean in intent_means.values()]
    return min(distances)

In [ ]:
# Compute minimum Mahalanobis distances for samples in test_embeddings and oos_test_embeddings
val_scores = [min_mahalanobis_for_sample(sample, intent_means, cov_inverse) for sample in val_embeddings]
oos_val_scores = [min_mahalanobis_for_sample(sample, intent_means, cov_inverse) for sample in oos_val_embeddings]

# True binary labels: 0 for in-domain and 1 for OOD
y_true = [0] * len(val_scores) + [1] * len(oos_val_scores)

# Combine the scores
y_scores = val_scores + oos_val_scores

# Compute AUPR
aupr = average_precision_score(y_true, y_scores)
aupr

In [ ]:
# Compute minimum Mahalanobis distances for samples in test_embeddings and oos_test_embeddings
test_scores = [min_mahalanobis_for_sample(sample, intent_means, cov_inverse) for sample in test_embeddings]
oos_test_scores = [min_mahalanobis_for_sample(sample, intent_means, cov_inverse) for sample in oos_test_embeddings]

# True binary labels: 0 for in-domain and 1 for OOD
y_true = [0] * len(test_scores) + [1] * len(oos_test_scores)

# Combine the scores
y_scores = test_scores + oos_test_scores

# Compute AUPR
aupr = average_precision_score(y_true, y_scores)
aupr


In [ ]:
auroc = roc_auc_score(y_true, y_scores)
auroc

In [ ]:
def get_predictions(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    real_labels = []
    with torch.no_grad():  # Disable gradient calculation
        for batch in data_loader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            outputs = model(input_ids, attention_mask) 
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.detach().cpu().numpy())  # Move preds to CPU and convert to numpy
            real_labels.extend(labels.detach().cpu().numpy())  # Move labels to CPU and convert to numpy
    return predictions, real_labels


In [ ]:
fine_model = torch.load(model_name)
fine_model = fine_model.to(device)
preds, true_labels = get_predictions(fine_model, test_dataloader)

# Now you can calculate the accuracy
accuracy = accuracy_score(true_labels, preds)
print(f"Classification Accuracy: {accuracy}")

In [ ]:
# import matplotlib.pyplot as plt

# # Set up the figure and axes
# plt.figure(figsize=(10, 6))

# # Plot the histograms
# plt.hist(val_scores, bins=50, alpha=0.5, label='In-domain')
# plt.hist(oos_val_scores, bins=50, alpha=0.5, label='Out-of-domain')

# # Add legend, title, and labels 
# plt.legend(loc='upper right')
# plt.title('Distribution of Minimum Mahalanobis Distances')
# plt.xlabel('Mahalanobis Distance')
# plt.ylabel('Number of Samples')
# plt.grid(True, which='both', linestyle='--', linewidth=0.5)
# plt.tight_layout()

# # Show the plot
# plt.show()


In [ ]:
# # Set up the figure and axes
# plt.figure(figsize=(10, 6))

# # Plot the histograms
# plt.hist(test_scores,bins=50, alpha=0.5, label='In-domain')
# plt.hist(oos_test_scores, bins=50, alpha=0.5, label='Out-of-domain')

# # Add legend, title, and labels
# plt.legend(loc='upper right')
# plt.title('Distribution of Minimum Mahalanobis Distances')
# plt.xlabel('Mahalanobis Distance')
# plt.ylabel('Number of Samples')
# plt.grid(True, which='both', linestyle='--', linewidth=0.5)
# plt.tight_layout()

# # Show the plot
# plt.show()


In [ ]:
# unique_labels = np.unique(test_labels)
# num_unique_labels = len(unique_labels)
 
# ood_label = "ood"
 
# # Combine embeddings
# combined_embeddings = np.vstack((test_embeddings, oos_test_embeddings))
 
# # Create labels for combined data
# combined_labels = np.concatenate((test_labels, np.array([ood_label] * len(oos_test_embeddings))))
 
# # TSNE
# tsne = TSNE(n_components=2, random_state=25)
# embeddings_2d = tsne.fit_transform(combined_embeddings)

In [ ]:
# plt.figure(figsize=(10, 8))
# unique_labels = np.unique(combined_labels)
# for label in unique_labels:
#     indices = np.where(combined_labels == label)[0]
#     if label == ood_label:
#         # Specific color for OOD
#         plt.scatter(embeddings_2d[indices, 0], embeddings_2d[indices, 1], label=label, color='black', s=0.5, alpha=0.7)
#     else:
#         plt.scatter(embeddings_2d[indices, 0], embeddings_2d[indices, 1], label=label, color='red', s=0.5, alpha=0.7)
# plt.title('2D t-SNE Plot of Embeddings (Baseline)')
# plt.legend()
# plt.show()

In [ ]:
# import optuna
# from transformers import AutoTokenizer, AutoModel

# # Assuming other necessary imports are already there

# def objective(trial):
#     # Optuna suggests hyperparameters
#     seed_value=42
#     random.seed(seed_value)
#     np.random.seed(seed_value)
#     torch.manual_seed(seed_value)
#     torch.cuda.manual_seed_all(seed_value)
#     lr = trial.suggest_categorical('lr', [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2])
#     num_epochs = 25
#     batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256])
#     training_losses = []
#     validation_losses = []
#     train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
#     # Model setup
#     tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
#     transformer_model = AutoModel.from_pretrained(pretrained_model_name)
#     model = TextClassifier(transformer_model, len(unique_intents))
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
    

#     # Loss function and optimizer
#     loss_function = nn.CrossEntropyLoss()
#     optimizer = Adam(model.parameters(), lr=lr)

#     best_val_loss = float('inf')
#     for epoch in range(num_epochs):
#         # Training Phase
#         model.train()  # Set the model to training mode
#         total_train_loss = 0
#         for batch in train_dataloader:
#             input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
#             optimizer.zero_grad()  # Zero the gradients
#             predictions = model(input_ids, attention_mask)  # Forward pass
#             loss = loss_function(predictions, labels) 
#             loss.backward()  # Backward pass
#             optimizer.step()  # Update weights
    
#             total_train_loss += loss.item()
        
#         avg_train_loss = total_train_loss / len(train_dataloader)
#         training_losses.append(avg_train_loss) 
    
#         # Validation Phase
#         model.eval()  # Set the model to evaluation mode
#         total_val_loss = 0
#         with torch.no_grad():  # Disable gradient calculations
#             for batch in val_dataloader:
#                 input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
#                 predictions = model(input_ids, attention_mask) 
#                 loss = loss_function(predictions, labels)  # Compute loss
#                 total_val_loss += loss.item()
#         avg_val_loss = total_val_loss / len(val_dataloader)
#         validation_losses.append(avg_val_loss)
#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             # Save the model
#             torch.save(model, model_name)
#             print(f"Epoch {epoch+1}/{num_epochs}: Lower validation loss found. Model saved.")
#     return best_val_loss
    
#     # trial.set_user_attr("training_losses", training_losses)
#     # trial.set_user_attr("validation_losses", validation_losses)
#     # model = torch.load(model_name)
#     # model.eval()  # Put the model in evaluation mode
#     # model = fine_model.to(device)
#     # fine_transformer = model.transformer
#     # train_embeddings = encode_sentences(fine_transformer, train_sentences)
#     # val_embeddings = encode_sentences(fine_transformer, val_sentences)
#     # oos_val_embeddings = encode_sentences(fine_transformer, oos_val_sentences)

#     # intent_means = {}
#     # for encoded_label in np.unique(encoded_train_labels):
#     #     # Find indices where the encoded label matches
#     #     indices = np.where(encoded_train_labels == encoded_label)[0]
        
#     #     # Calculate the mean embedding for the current intent
#     #     intent_embeddings = train_embeddings[indices]
#     #     intent_mean = np.mean(intent_embeddings, axis=0)
        
#     #     # Use the encoded label as the dictionary key
#     #     intent_means[encoded_label] = intent_mean
#     # covariance = np.cov(train_embeddings, rowvar=False)
#     # cov_inverse = inv(covariance)
#     # val_scores = [min_mahalanobis_for_sample(sample, intent_means, cov_inverse) for sample in val_embeddings]
#     # oos_val_scores = [min_mahalanobis_for_sample(sample, intent_means, cov_inverse) for sample in oos_val_embeddings]

#     # # True binary labels: 0 for in-domain and 1 for OOD
#     # y_true = [0] * len(val_scores) + [1] * len(oos_val_scores)

#     # # Combine the scores
#     # y_scores = val_scores + oos_val_scores

#     # # Compute AUPR
#     # aupr = average_precision_score(y_true, y_scores)
    
#     # return aupr

# # Create a study object and optimize the objective function
# study = optuna.create_study(direction='minimize',  study_name='ce_loss_CLINC150_min_val', storage='sqlite:///desperate.db', load_if_exists= True)
# study.optimize(objective, n_trials=35)  # n_trials is the number of iterations

# # Get the best parameters
# best_params = study.best_params
# print("Best parameters:", best_params)

